#  Training a CNN based on the extracted Spectrograms

In [10]:
import matplotlib.pyplot as plt
from python_speech_features import mfcc, ssc, logfbank
import pandas as pd
import os
import numpy as np
import soundfile
from scipy import signal
import librosa
import matplotlib.pyplot as plt
import librosa.display
from tensorflow import keras

# Loading the data

In [3]:
URBAN_SOUND_DIR = "/media/nvidia/ROMULO'S/urban_sound_challenge/"

In [5]:
temp = pd.read_json(URBAN_SOUND_DIR+"test_mel_spectrogram_transformed.json")

In [6]:
x = np.array(temp.feature.tolist())

In [7]:
x_features = []
for i in range(x.shape[0]):
    _x_array = np.array([np.array(m) for m in x[i]])
    x_features.append(_x_array[:,:99])
x_features= np.array(x_features)

In [8]:
x_features.shape

(3297, 23, 99)

In [9]:
encoded_classes = {'siren' :0,
 'street_music' :1,
 'engine_idling':2,
 'jackhammer':3,
 'car_horn':4,
 'drilling':5,
 'dog_bark':6,
 'air_conditioner':7,
 'children_playing':8,
 'gun_shot':9}

In [21]:
def find_class_from_val(x):
    for label, code in encoded_classes.items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
        if code == x:
            return(label)

# Loading NN Architecture

In [11]:
with open("./trained_models/hdf5/spectrogram_simple_nn_urban_sound_model.json", 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./trained_models/hdf5/spectrogram_simple_nn_urban_sound_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [16]:
y_prediction = loaded_model.predict(x_features,verbose=0)

In [13]:
y_prediction.shape

(3297, 10)

In [17]:
test_df = pd.read_csv(URBAN_SOUND_DIR+'test.csv')

In [18]:
test_df['prediction'] = y_prediction.argmax(axis=-1)

In [22]:
test_df['Class'] = test_df['prediction'].apply(find_class_from_val)

In [23]:
test_df.head()

,ID,prediction,Class
0,5,3,jackhammer
1,7,6,dog_bark
2,8,6,dog_bark
3,9,9,gun_shot
4,13,3,jackhammer


In [24]:
test_df.to_csv(URBAN_SOUND_DIR+'test_prediction_simple_nn.csv')